# RoBERTa를 이용한 한국어 자연어추론(NLI)
- 사전학습 모델 : KLUE-RoBERTa (MODU, CC-100-Kor, NAMUWIKI, NEWSCRAWL, PETITION)
- 데이터 : KLUE-NLI (WIKITREE, POLICY, WIKINEWS, WIKIPEDIA, NSMC and AIRBNB)

# 사전 준비

In [ ]:
!pip install transformers
!pip install datasets

**KLUE-NLI 데이터 불러오기**

In [2]:
from datasets import load_dataset

datasets = load_dataset("klue", "nli")

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 3000
    })
})

In [4]:
# label 0: entailment(함의) / 1: neutral(중립) / 2: contradiction(모순)
print(datasets["train"][0])
print(datasets["validation"][0])

{'guid': 'klue-nli-v1_train_00000', 'source': 'NSMC', 'premise': '힛걸 진심 최고다 그 어떤 히어로보다 멋지다', 'hypothesis': '힛걸 진심 최고로 멋지다.', 'label': 0}
{'guid': 'klue-nli-v1_dev_00000', 'source': 'airbnb', 'premise': '흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.', 'hypothesis': '어떤 방에서도 흡연은 금지됩니다.', 'label': 2}


**KLUE-RoBERTa 모델과 토크나이저 불러오기**

In [5]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("klue/roberta-base")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [6]:
model.config

RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}

# 토크나이징, 데이터 구축

**스페셜 토큰 확인**

In [20]:
for i in range (10):
    print("index : ",i," =  tokens : ",tokenizer.decode(i))

index :  0  =  tokens :  [CLS]
index :  1  =  tokens :  [PAD]
index :  2  =  tokens :  [SEP]
index :  3  =  tokens :  [UNK]
index :  4  =  tokens :  [MASK]
index :  5  =  tokens :  !
index :  6  =  tokens :  "
index :  7  =  tokens :  #
index :  8  =  tokens :  $
index :  9  =  tokens :  %


**[CLS] 전제 [SEP] 가설 [SEP] [PAD]...**

In [23]:
import torch
from torch.utils.data import Dataset

In [50]:
class NLIDataset(Dataset):
    def __init__(self, data, max_len=64):  # 데이터셋의 전처리를 해주는 부분
        self._data = data
        self.max_len = max_len
        self.bos = tokenizer.bos_token      # [CLS]
        self.eos = tokenizer.eos_token      # [SEP]
        self.pad = tokenizer.pad_token      # [PAD]
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        index = self._data[idx]

        p = index["premise"]  # 전제
        p_toked = self.tokenizer.tokenize(self.bos + p + self.eos)      # [CLS] 전제 [SEP]
        p_len = len(p_toked)

        h = index["hypothesis"]  # 가설
        h_toked = self.tokenizer.tokenize(h + self.eos)      # 가설 [SEP]
        h_len = len(p_toked)

        # 전제 + 가설 길이가 최대길이보다 클때
        if p_len + h_len > self.max_len:
            h_len = self.max_len - p_len        # 가설의 길이 = 최대길이 - 전제길이

            if p_len <= 0:       # 전제의 길이가 너무 길어 전제만으로 최대 길이를 초과 한다면
                p_toked = p_toked[-(int(self.max_len / 2)) :]   # 전제길이를 최대길이의 반으로 
                p_len = len(p_toked)
                h_len = self.max_len - p_len              # 답변의 길이를 최대길이 - 전제길이
                
            h_toked = h_toked[:h_len]
            h_len = len(h_toked)

        # 전제 + 가설 토큰을 index로 변환   
        token_ids = self.tokenizer.convert_tokens_to_ids(p_toked + h_toked)

        # 최대 길이만큼 padding
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # attention_mask(어텐션마스크) = 전제 + 가설 길이 1 + 나머지(패딩) 0
        attention_mask = [1]*(p_len + h_len) + [0]*(self.max_len - p_len - h_len)

        # token_type_ids(세그먼트 정보) = 전제 0 + 가설 1 + 나머지 0
        token_type_ids = [0]*p_len + [1]*h_len + [0]*(self.max_len - p_len - h_len)

        # label = 0: entailment(함의) / 1: neutral(중립) / 2: contradiction(모순)
        label = index["label"]

        # 질문 + 답변, 어텐션마스크, 세그먼트 정보, 답변
        return (token_ids, attention_mask, token_type_ids, label)

**데이터셋 구축** <br>
구성 : (token_ids, attention_mask, token_type_ids, label)

In [54]:
# 훈련 데이터셋
train_dataset = NLIDataset(datasets["train"])

for n in range(5):
    print("train_dataset[",n,"]")
    print("token_ids      : ", train_dataset[n][0])
    print("attention_mask : ", train_dataset[n][1])
    print("token_type_ids : ", train_dataset[n][2])
    print("label          : ", train_dataset[n][3],"\n")

train_dataset[ 0 ]
token_ids      :  [0, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2, 3, 7254, 3841, 2200, 11980, 2062, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
token_type_ids :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
label          :  0 

train_dataset[ 1 ]
token_ids      :  [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 1525, 2062, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [55]:
# 검증 데이터셋
val_dataset = NLIDataset(datasets["validation"])

for n in range(5):
    print("val_dataset[",n,"]")
    print("token_ids      : ", val_dataset[n][0])
    print("attention_mask : ", val_dataset[n][1])
    print("token_type_ids : ", val_dataset[n][2])
    print("label          : ", val_dataset[n][3],"\n")

val_dataset[ 0 ]
token_ids      :  [0, 25313, 2377, 2031, 2073, 20812, 2116, 1513, 2259, 1129, 24094, 20812, 27135, 9753, 2052, 3662, 11800, 18, 2, 3711, 1129, 27135, 2119, 9753, 2073, 5040, 3598, 3606, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
token_type_ids :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
label          :  2 

val_dataset[ 1 ]
token_ids      :  [0, 3633, 2211, 2052, 3655, 3704, 31302, 5153, 2530, 4087, 4671, 2371, 2062, 18, 2, 3633, 2211, 2052, 3655, 3704, 31302, 5153, 2530, 2052, 1039, 2886, 2062, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 